In [227]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta


In [228]:
data = pd.read_csv('data.csv')

In [229]:
# Parse the JSON data in the 'contracts' list

for index, row in data.iterrows():
    if pd.notna(row['contracts']):
        try:
            contracts_data = json.loads(row['contracts'])
        except (json.JSONDecodeError, TypeError) as e:
            print(f"Error parsing JSON for id {row['id']}: {e}")



In [230]:
contracts_data

[{'contract_id': 18410,
  'bank': '004',
  'summa': 49164700,
  'loan_summa': 0,
  'claim_date': '03.04.2019',
  'claim_id': 18410,
  'contract_date': '03.04.2019'},
 {'contract_id': '',
  'bank': '004',
  'summa': '',
  'loan_summa': '',
  'claim_date': '22.02.2021',
  'claim_id': 43136,
  'contract_date': ''},
 {'contract_id': '',
  'bank': '062',
  'summa': '',
  'loan_summa': '',
  'claim_date': '14.02.2022',
  'claim_id': 3304534,
  'contract_date': ''},
 {'contract_id': 3304674,
  'bank': '062',
  'summa': 405000000,
  'loan_summa': 217320883,
  'claim_date': '14.02.2022',
  'claim_id': 3304674,
  'contract_date': '14.02.2022'},
 {'contract_id': '',
  'bank': '004',
  'summa': '',
  'loan_summa': '',
  'claim_date': '18.02.2022',
  'claim_id': 62750,
  'contract_date': ''},
 {'contract_id': '',
  'bank': '014',
  'summa': '',
  'loan_summa': '',
  'claim_date': '31.05.2023',
  'claim_id': 'F00895453',
  'contract_date': ''}]

In [231]:
df_normalized = data['contracts'].explode()


In [232]:
# Parse the 'contracts' column
data['contracts'] = data['contracts'].apply(lambda x: json.loads(x) if pd.notna(x) else [])



In [233]:
data = data.explode('contracts').reset_index()
data

,index,id,application_date,contracts
0,0,2925210.0,2024-02-12 19:22:46.652000+00:00,NaN
1,1,2925211.0,2024-02-12 19:24:29.135000+00:00,"{'contract_id': 522530, 'bank': '003', 'summa'..."
2,1,2925211.0,2024-02-12 19:24:29.135000+00:00,"{'contract_id': '', 'bank': '014', 'summa': ''..."
3,1,2925211.0,2024-02-12 19:24:29.135000+00:00,"{'contract_id': '', 'bank': '014', 'summa': ''..."
4,1,2925211.0,2024-02-12 19:24:29.135000+00:00,"{'contract_id': '', 'bank': '014', 'summa': ''..."
...,...,...,...,...
10064,995,2926205.0,2024-02-13 06:09:54.210000+00:00,"{'contract_id': '', 'bank': '014', 'summa': ''..."
10065,996,2926206.0,2024-02-13 06:09:54.306000+00:00,NaN
10066,997,2926207.0,2024-02-13 06:09:55.661000+00:00,NaN
10067,998,2926208.0,2024-02-13 06:09:57.024000+00:00,NaN


In [234]:
exploded_data = pd.json_normalize(data['contracts'])

In [235]:
df_final = pd.concat([data.drop(['contracts'], axis=1), 
                     exploded_data.apply(pd.Series)], axis=1)

In [236]:
df_final

,index,id,application_date,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date
0,0,2925210.0,2024-02-12 19:22:46.652000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2925211.0,2024-02-12 19:24:29.135000+00:00,522530,003,500000000,0,13.02.2020,609965,17.02.2020
2,1,2925211.0,2024-02-12 19:24:29.135000+00:00,,014,,,28.08.2020,F00013731,
3,1,2925211.0,2024-02-12 19:24:29.135000+00:00,,014,,,08.10.2020,F00021301,
4,1,2925211.0,2024-02-12 19:24:29.135000+00:00,,014,,,25.11.2020,F00037907,
...,...,...,...,...,...,...,...,...,...,...
10064,995,2926205.0,2024-02-13 06:09:54.210000+00:00,,014,,,31.05.2023,F00895453,
10065,996,2926206.0,2024-02-13 06:09:54.306000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10066,997,2926207.0,2024-02-13 06:09:55.661000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10067,998,2926208.0,2024-02-13 06:09:57.024000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [237]:
# Calculate the date 180 days ago from the latest date from max date in dataset

df_final['claim_date'] = pd.to_datetime(df_final['claim_date'], format='%d.%m.%Y', errors='coerce')
latest_date = df_final['claim_date'].max()
date_180_days_ago = latest_date - timedelta(days=180)

In [238]:
def count_unique_claims(group):
    recent_claims = group[group['claim_date'] > date_180_days_ago]
    return len(recent_claims['claim_id'].unique())

claims_count = df_final.groupby('claim_id').apply(count_unique_claims).reset_index(name='tot_claim_cnt_180d')

df_final = df_final.merge(claims_count, on='claim_id', how='left')

In [246]:
df_final['tot_claim_cnt_180d'].replace(0, -3, inplace=True)
df_final['tot_claim_cnt_180d'].value_counts()

tot_claim_cnt_180d
-3.0    5346
 1.0    3874
Name: count, dtype: int64

In [239]:
claims_count

,claim_id,tot_claim_cnt_180d
0,2401,0
1,2414,0
2,3056,0
3,3501,0
4,3730,0
...,...,...
2524,o326569,0
2525,o339901,0
2526,o339902,0
2527,s020a699b1,1


In [240]:
grouped_tot_claim_cnt_180d = df_final.groupby('id')['tot_claim_cnt_180d'].count()
grouped_tot_claim_cnt_180d.replace(0, -3)

id
2925210.0    -3
2925211.0    82
2925212.0    -3
2925213.0    82
2925214.0    -3
             ..
2926205.0     6
2926206.0    -3
2926207.0    -3
2926208.0    -3
2926209.0    -3
Name: tot_claim_cnt_180d, Length: 1000, dtype: int64

In [241]:
df_final['loan_summa'] = pd.to_numeric(df_final['loan_summa'], errors='coerce')
excluded_banks = ['LIZ', 'LOM', 'MKO', 'SUG']

# Function to calculate the sum of loan_summa for eligible loans

def calc_disb_bank_loan_wo_tbc(group):
    eligible_loans = group[
        (group['bank'].notna()) & 
        (~group['bank'].isin(excluded_banks)) & 
        (group['contract_date'].notna()) &
        (group['loan_summa'].notna())
    ]
    return eligible_loans['loan_summa'].sum()

loan_sums = df_final.groupby('claim_id').apply(calc_disb_bank_loan_wo_tbc).reset_index(name='disb_bank_loan_wo_tbc')
df_final = df_final.merge(loan_sums, on='claim_id', how='left')
df_final['disb_bank_loan_wo_tbc'] = df_final['disb_bank_loan_wo_tbc'].apply(
    lambda x: -3 if pd.isna(x) else (-1 if x == 0 else x)
)


In [242]:
df_final['disb_bank_loan_wo_tbc'].value_counts()

disb_bank_loan_wo_tbc
-1.000000e+00    8879
-3.000000e+00     849
 2.654550e+09       6
 1.935156e+10       6
 3.546808e+09       6
                 ... 
 3.508718e+08       2
 1.120373e+09       2
 7.546077e+08       2
 7.539974e+08       2
 7.815963e+09       2
Name: count, Length: 115, dtype: int64

In [247]:
df_final['contract_date'] = pd.to_datetime(df_final['contract_date'], format='%d.%m.%Y', errors='coerce')
df_final['summa'] = pd.to_numeric(df_final['summa'], errors='coerce')
application_date = df_final['contract_date'].max()

# Function to calculate days since last loan

def calc_days_since_last_loan(group):
    valid_loans = group[(group['summa'].notna()) & (group['summa'] > 0) & (group['contract_date'].notna())]
    if valid_loans.empty:
        return None
    last_loan_date = valid_loans['contract_date'].max()
    return (application_date - last_loan_date).days

days_since_last_loan = df_final.groupby('claim_id').apply(calc_days_since_last_loan).reset_index(name='day_sinlastloan')
df_final = df_final.merge(days_since_last_loan, on='claim_id', how='left')
df_final['day_sinlastloan'] = df_final['day_sinlastloan'].apply(
    lambda x: -3 if pd.isna(x) else (-1 if x is None else x)
)


In [249]:
df_csv = df_final[['id', 'application_date', 'tot_claim_cnt_180d', 'disb_bank_loan_wo_tbc', 'day_sinlastloan']]

In [250]:
df_csv.to_csv('contract_features.csv')